In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("DataPreprocessing") \
    .master("spark://localhost:7077") \
    .getOrCreate()

24/02/15 01:56:37 WARN Utils: Your hostname, Bogdans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.75 instead (on interface en0)
24/02/15 01:56:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/15 01:56:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
